In [1]:
from __future__ import division
import os
import sys

import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline

import gffutils as gfu

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## 1b)
#### Create a database using gffutils (you can install gffutils via pip). Remember to only make a database once!

In [2]:
%%time
db = gfu.create_db('saccharomyces_cerevisiae.gff', dbfn='sacCer.db')

CPU times: user 1.93 s, sys: 8.69 ms, total: 1.94 s
Wall time: 2.67 s
